In [2]:
import torch
import matplotlib.pyplot as plt
from torch import nn
from torch.nn import functional as F

In [3]:
batch_size, num_steps = 32, 35

In [6]:
!curl --output pap.txt https://www.gutenberg.org/files/1342/1342-0.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  5  780k    5 41203    0     0  20601      0  0:00:38  0:00:02  0:00:36 16177
 54  780k   54  422k    0     0   140k      0  0:00:05  0:00:03  0:00:02  110k
100  780k  100  780k    0     0   260k      0  0:00:03  0:00:03 --:--:--  197k


In [18]:
import re
def read_text(text_name='pap.txt'):
    with open(text_name, 'r', encoding='utf-8') as text_input:
        lines = text_input.readlines()
    
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines] # only alphabets

In [16]:
text_input = read_text()

In [17]:
text_input[:5]

['the project gutenberg ebook of pride and prejudice by jane austen',
 '',
 'this ebook is for the use of anyone anywhere in the united states and',
 'most other parts of the world at no cost and with almost no restrictions',
 'whatsoever you may copy it give it away or re use it under the terms']

In [25]:
def tokenize(lines, token_type='word'):
    if token_type == 'char':
        return [list(line) for line in lines]
    elif token_type == 'word':
        return [line.split() for line in lines]
    else:
        'Wrong token type.'

In [27]:
import collections

def count_corpus(tokens):
    return collections.Counter(tokens)

In [58]:
# Defined in file: ./chapter_recurrent-neural-networks/text-preprocessing.md
class Vocab:
    def __init__(self, tokens=None,min_freq=0):
        if tokens == None:
            tokens = []
        
        counter = collections.Counter(tokens)
        
        self._token_freqs = sorted(counter.items(),key=lambda x:x[1], reverse=True)
        self.idx_to_token = ['<unk>']
        self.token_to_idx = { token:idx for idx, token in enumerate(self.idx_to_token)}
        
        for token, freq in self._token_freqs:
            if freq > min_freq:
                break
            elif token not in self.idx_to_token:
                print(token + "adding")
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token)-1
            else:
                print(token + " found already")
    
    def __len__(self):
        return len(self.idx_to_token)
    
    def __getitem__(self, tokens):
        if isinstance(tokens, list):
            return_list = []
            for t in tokens:
                for token in t:
                    if self.token_to_idx[token]:
                        return_list.append(self.token_to_idx[token])
                    else:
                        return_list.append(self.unk())
            return return_list
        else:
            if self.token_to_idx[tokens]:
                return self.token_to_idx[tokens]
            else:
                return self.unk()
            
    def to_tokens(self, indices):
        return [self.idx_to_token[index] for index in indices]

    @property
    def unk(self):  # Index for the unknown token
        return 0

    @property
    def token_freqs(self):  # Index for the unknown token
        return self._token_freqs


In [61]:
tokens = tokenize(text_input)
vocab = Vocab(text_input)
    

In [65]:
tokens
for i in vocab.token_freqs:
    print(i)
    if i[0] == 'the':
        print(i)
# print(vocab.token_freqs)
# vocab.token_to_idx

# the error is that we are using the entire sentece for creating token
# problem in tokenize

('', 2427)
('chapter', 122)
('elizabeth', 8)
('them', 7)
('it', 7)
('her', 6)
('himself', 4)
('town', 3)
('longbourn', 3)
('this ebook is for the use of anyone anywhere in the united states and', 2)
('daughters', 2)
('with', 2)
('him', 2)
('darcy', 2)
('you', 2)
('meryton', 2)
('love', 2)
('bennet', 2)
('dear sir', 2)
('happiness', 2)
('character', 2)
('lizzy', 2)
('daughter', 2)
('away', 2)
('understood', 2)
('derbyshire', 2)
('room', 2)
('strangers', 2)
('wife', 2)
('case', 2)
('continued', 2)
('days', 2)
('jane', 2)
('opinion', 2)
('acquaintance', 2)
('again', 2)
('section information about donations to the project gutenberg', 2)
('literary archive foundation', 2)
('the project gutenberg ebook of pride and prejudice by jane austen', 1)
('most other parts of the world at no cost and with almost no restrictions', 1)
('whatsoever you may copy it give it away or re use it under the terms', 1)
('of the project gutenberg license included with this ebook or online at', 1)
('www gutenberg o

('lieutenant s commission in the shire she had been watching him', 1)
('the last hour she said as he walked up and down the street and', 1)
('had mr wickham appeared kitty and lydia would certainly have', 1)
('continued the occupation but unluckily no one passed windows now', 1)
('except a few of the officers who in comparison with the', 1)
('stranger were become stupid disagreeable fellows some of', 1)
('them were to dine with the phillipses the next day and their', 1)
('aunt promised to make her husband call on mr wickham and give', 1)
('him an invitation also if the family from longbourn would come', 1)
('in the evening this was agreed to and mrs phillips protested', 1)
('that they would have a nice comfortable noisy game of lottery', 1)
('tickets and a little bit of hot supper afterwards the prospect', 1)
('of such delights was very cheering and they parted in mutual', 1)
('good spirits mr collins repeated his apologies in quitting the', 1)
('room and was assured with unwearying ci

('not object to it why should we', 1)
('her not objecting does not justify him it only shows her', 1)
('being deficient in something herself sense or feeling', 1)
('well cried elizabeth have it as you choose he shall be', 1)
('mercenary and she shall be foolish', 1)
('no lizzy that is what i do not choose i should be sorry', 1)
('you know to think ill of a young man who has lived so long in', 1)
('oh if that is all i have a very poor opinion of young men who', 1)
('live in derbyshire and their intimate friends who live in', 1)
('hertfordshire are not much better i am sick of them all thank', 1)
('heaven i am going to morrow where i shall find a man who has not', 1)
('one agreeable quality who has neither manner nor sense to', 1)
('recommend him stupid men are the only ones worth knowing after', 1)
('all', 1)
('take care lizzy that speech savours strongly of', 1)
('disappointment', 1)
('before they were separated by the conclusion of the play she had', 1)
('the unexpected happiness of a

('unconcern oh but there were two or three much uglier in the', 1)
('shop and when i have bought some prettier coloured satin to trim', 1)
('it with fresh i think it will be very tolerable besides it', 1)
('will not much signify what one wears this summer after the', 1)
('shire have left meryton and they are going in a fortnight', 1)
('are they indeed cried elizabeth with the greatest', 1)
('satisfaction', 1)
('they are going to be encamped near brighton and i do so want', 1)
('papa to take us all there for the summer it would be such a', 1)
('delicious scheme and i dare say would hardly cost anything at', 1)
('all mamma would like to go too of all things only think what a', 1)
('miserable summer else we shall have', 1)
('yes thought elizabeth that would be a delightful scheme', 1)
('indeed and completely do for us at once good heaven brighton', 1)
('and a whole campful of soldiers to us who have been overset', 1)
('already by one poor regiment of militia and the monthly balls of', 1)


('other how am i ever to pay him', 1)
('money my uncle cried jane what do you mean sir', 1)
('i mean that no man in his senses would marry lydia on so slight', 1)
('a temptation as one hundred a year during my life and fifty', 1)
('after i am gone', 1)
('that is very true said elizabeth though it had not occurred', 1)
('to me before his debts to be discharged and something still to', 1)
('remain oh it must be my uncle s doings generous good man i', 1)
('am afraid he has distressed himself a small sum could not do all', 1)
('this', 1)
('no said her father wickham s a fool if he takes her with a', 1)
('farthing less than ten thousand pounds i should be sorry to', 1)
('think so ill of him in the very beginning of our relationship', 1)
('ten thousand pounds heaven forbid how is half such a sum to be', 1)
('repaid', 1)
('mr bennet made no answer and each of them deep in thought', 1)
('continued silent till they reached the house their father then', 1)
('went on to the library to write and t

('yours sincerely etc', 1)
('miss bingley s congratulations to her brother on his approaching', 1)
('marriage were all that was affectionate and insincere she wrote', 1)
('even to jane on the occasion to express her delight and repeat', 1)
('all her former professions of regard jane was not deceived but', 1)
('she was affected and though feeling no reliance on her could', 1)
('not help writing her a much kinder answer than she knew was', 1)
('deserved', 1)
('the joy which miss darcy expressed on receiving similar', 1)
('information was as sincere as her brother s in sending it four', 1)
('sides of paper were insufficient to contain all her delight and', 1)
('all her earnest desire of being loved by her sister', 1)
('before any answer could arrive from mr collins or any', 1)
('congratulations to elizabeth from his wife the longbourn family', 1)
('heard that the collinses were come themselves to lucas lodge the', 1)
('reason of this sudden removal was soon evident lady catherine', 1)
('h

In [36]:
def load_corpus(max_tokens=-1, text_input=text_input):
    tokens = tokenize(text_input)
    vocab = Vocab(text_input)
    
    corpus = [vocab[token] for line in tokens for token in line]
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    return corpus, vocab
    

In [37]:
corpus, vocab = load_corpus()

KeyError: 'the'